In [1]:
import psycopg2
from psycopg2 import sql
import numpy as np
import pandas as pd
from psycopg2.extensions import adapt, register_adapter, AsIs
from sqlalchemy import create_engine, text
import getpass

In [2]:
file_paths = ['2018_Median_Rooms_by_Tenure.csv', 
              '2019_Median_Rooms_by_Tenure.csv',
              '2020_Median_Rooms_by_Tenure.csv', 
              '2021_Median_Rooms_by_Tenure.csv', 
              '2022_Median_Rooms_by_Tenure.csv']

dataframes = [pd.read_csv(file) for file in file_paths]
Median_Rooms_by_Tenure = pd.concat(dataframes, ignore_index=True)
merged_file_path = 'Median_Rooms_by_Tenure.csv'
Median_Rooms_by_Tenure.to_csv(merged_file_path, index=False)
merged_file_path

'Median_Rooms_by_Tenure.csv'

In [3]:
database = "casestudyf24t03"
user     = input("Type username (pawprint) and hit enter: ")
password = getpass.getpass("Type password and hit enter: ")

connection = psycopg2.connect(database = database,
                              user     = user,
                              host     = 'pgsql.dsa.lan',
                              password = password)

Type username (pawprint) and hit enter: jsmm8
Type password and hit enter: ········


In [4]:
with connection, connection.cursor() as cursor:
    cursor.execute("DROP TABLE IF EXISTS median_rooms_by_tenure;")
    

    cursor.execute(
        '''
        CREATE TABLE median_rooms_by_tenure (
    geo_id TEXT,
    block_group TEXT,
    census_tract TEXT,
    county TEXT,
    state TEXT,
    est_median_rooms_total INTEGER,
    moe_median_rooms_total INTEGER,
    est_median_rooms_owner INTEGER,
    moe_median_rooms_owner INTEGER,
    est_median_rooms_renter INTEGER,
    moe_median_rooms_renter INTEGER,
    year INTEGER,
    PRIMARY KEY (geo_id, year)
    );
    '''
    )

In [5]:
mypasswd = getpass.getpass("Enter your database password: ")
username = 'jsmm8'
host = 'pgsql.dsa.lan'
database = 'casestudyf24t03'

conn_string = f"postgresql+psycopg2://{username}:{mypasswd}@{host}/{database}"
engine = create_engine(conn_string)

median_rooms_by_tenure = 'Median_Rooms_by_Tenure.csv'

chunk_size = 500

try:
    for chunk in pd.read_csv(median_rooms_by_tenure, chunksize=chunk_size):
        chunk.to_sql('median_rooms_by_tenure', engine, if_exists='append', index=False)
        print(f"Uploaded a chunk of {len(chunk)} records to the database.")    
    print("CSV data uploaded successfully.")

except Exception as e:
    print(f"An error occurred: {e}")

Enter your database password: ········
Uploaded a chunk of 500 records to the database.
Uploaded a chunk of 500 records to the database.
Uploaded a chunk of 500 records to the database.
Uploaded a chunk of 500 records to the database.
Uploaded a chunk of 500 records to the database.
Uploaded a chunk of 500 records to the database.
Uploaded a chunk of 500 records to the database.
Uploaded a chunk of 500 records to the database.
Uploaded a chunk of 500 records to the database.
Uploaded a chunk of 500 records to the database.
Uploaded a chunk of 500 records to the database.
Uploaded a chunk of 500 records to the database.
Uploaded a chunk of 500 records to the database.
Uploaded a chunk of 500 records to the database.
Uploaded a chunk of 500 records to the database.
Uploaded a chunk of 500 records to the database.
Uploaded a chunk of 500 records to the database.
Uploaded a chunk of 500 records to the database.
Uploaded a chunk of 500 records to the database.
Uploaded a chunk of 500 record

In [6]:
connection.close()

In [7]:
grant_privileges_query = "GRANT ALL PRIVILEGES ON TABLE median_rooms_by_tenure TO ypd5yb, remcmf, sgdky;"

with engine.connect() as connection:
        connection.execute(text(grant_privileges_query))
        print("Privileges granted successfully.")

Privileges granted successfully.


In [8]:
select_query = "SELECT * FROM median_rooms_by_tenure LIMIT 10;" 

with engine.connect() as connection:
    result = connection.execute(text(select_query))
    df = pd.DataFrame(result.fetchall(), columns=result.keys())
print("Query executed successfully.")
df

Query executed successfully.


,geo_id,block_group,census_tract,county,state,est_median_rooms_total,moe_median_rooms_total,est_median_rooms_owner,moe_median_rooms_owner,est_median_rooms_renter,moe_median_rooms_renter,year
0,1500000US290019501001,Block Group 1,Census Tract 9501,Adair County,Missouri,7,0,7,0,7,3,2018
1,1500000US290019501002,Block Group 2,Census Tract 9501,Adair County,Missouri,7,1,7,1,6,1,2018
2,1500000US290019501003,Block Group 3,Census Tract 9501,Adair County,Missouri,6,1,6,0,5,1,2018
3,1500000US290019502001,Block Group 1,Census Tract 9502,Adair County,Missouri,6,0,6,0,4,1,2018
4,1500000US290019502002,Block Group 2,Census Tract 9502,Adair County,Missouri,7,0,7,0,6,3,2018
5,1500000US290019502003,Block Group 3,Census Tract 9502,Adair County,Missouri,6,1,7,1,4,2,2018
6,1500000US290019503001,Block Group 1,Census Tract 9503,Adair County,Missouri,6,1,6,1,4,1,2018
7,1500000US290019503002,Block Group 2,Census Tract 9503,Adair County,Missouri,5,1,6,1,4,1,2018
8,1500000US290019503003,Block Group 3,Census Tract 9503,Adair County,Missouri,5,1,6,0,4,1,2018
9,1500000US290019503004,Block Group 4,Census Tract 9503,Adair County,Missouri,3,0,8,3,3,0,2018
